# 4_masking結果とc2sファイルの結合
c2sは文字列の辞書順でソートされている。0=>1=>11=>12...

globにも文字列の辞書順でそ～とかけることでマッチする

2つを内部結合させたdfを作る

その後joinさせる

In [60]:
import pandas as pd
import numpy as np
import glob
from pprint import pprint
from typing import List

In [61]:
MASKING_DATA_PATH = "../../data/3_masking_mini/maskedfiles" # 入力データセット
STYLES = ["training","validation","test"] 
C2S_DIR_PATH = "../../data/2_convert_c2s_mini"
C2S_PATHS = [C2S_DIR_PATH+"/java-small_raw.train.c2s",C2S_DIR_PATH+"/java-small_raw.val.c2s",C2S_DIR_PATH+"/java-small_raw.test.c2s"]

In [62]:
# c2s取り出し処理を先にやってidxを記録しておく
idxs= []
masked_methods = []
for c2s_path in C2S_PATHS:
    df = pd.read_table(c2s_path,header=None,names=["c2s_method"],skip_blank_lines=False)
df = df.dropna(how='all')
df = df[df["c2s_method"].str.contains('[a-z]+')]# 空行列の削除
df

,c2s_method
0,"setup void,Void0|Mth|Nm1,METHOD_NAME void,Void..."
1,"test|append|to|empty|file target,Nm3|Cal|Fld3|..."
2,"test|open|read|dir|with|child instream,Nm0|As|..."
3,"test|open|file|twice read,Nm2|Cal3|Cal|Ex|Bk|E..."
4,"test|sequential|read instream,Nm0|Cal2|Cal|Nm3..."
5,"test|rename|new|file|same|dir describe,Nm2|Cal..."
6,"test|rename|nonexistent|file path,Cls0|VDE|Ex|..."
7,test|rename|file|over|existing|file METHOD_NAM...
8,"test|rename|dir|into|existing|dir string,Cls0|..."
9,"test|rename|file|nonexistent|dir false,BoolEx3..."


In [80]:
OUTPUT_PATH = "../../data/4_combine_mini/"
TRUE_STYLES = ["train","val","test"]

# masked?df = 
for style, c2s_path, true_style in zip(STYLES,C2S_PATHS,TRUE_STYLES):
    # masked_methods
    idxs= []
    masked_methods = []
    filepaths = sorted(glob.glob(f"{MASKING_DATA_PATH}/{style}/*.java"))
    for path in filepaths:
        idx = int(path.split("/")[-1].replace(".java","") )# 数字のindex
        idxs.append(idx)
        with open(path) as f:
            masked_methods.append(f.read())
    masked_df = pd.DataFrame(index = idxs, data={"masked_method": masked_methods})         

    # c2s_methods
    c2s_df = pd.read_table(c2s_path,header=None,names=["c2s_method"],skip_blank_lines=False)
    c2s_df = c2s_df[c2s_df["c2s_method"].str.contains('[a-z]+')]# 空行列の削除
    result_df = c2s_df.join(masked_df, how='inner')
    result_df["train_str"] = result_df["masked_method"] + " $$$ " + result_df["c2s_method"]

    result_df.to_csv(f"{OUTPUT_PATH}dataframe/{style}.csv")
    with open(OUTPUT_PATH+"input_c2s/"+true_style+".c2s", "w") as f:
        li = result_df["train_str"].values.tolist()
        li = map(lambda x: x + "\n", li)
        f.writelines(li)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 59
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   c2s_method     58 non-null     object
 1   masked_method  58 non-null     object
 2   train_str      58 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB
